In [2]:
import os
import time


In [20]:
try:
    %load_ext autotime
except:
    !pip install -q ipython-autotime
    %load_ext autotime

time: 888 µs (started: 2024-04-24 20:12:41 +03:00)


In [13]:
!pip install jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.2/407.2 kB 4.7 MB/s eta 0:00:00a 0:00:01


In [25]:
!pip install faiss-cpu # для gpu тут другая установка, faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 5.1 MB/s eta 0:00:0000:0100:01
time: 5.69 s (started: 2024-04-24 20:16:02 +03:00)


In [26]:
!pip install chromadb # вот этот опционально, пока что с ним запустить не удалось, и скачивается долго

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 769.0 kB/s eta 0:00:00 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 140.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.2/221.2 kB 178.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.5/528.5 kB 195.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 129.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 177.2 kB/s eta 0:00:0000:010:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 158.5 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 129.9 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 191.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 151.4 

In [4]:
CREDS = 

In [7]:
import json
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

In [9]:
PATH_DATASET = 'texts_rag.csv'
JSON_SAVE_PATH = "texts_all.json"

In [14]:
%%time
# Определяем функцию извлечения метаданных.
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["source"] = record.get("url")
    # metadata["Заголовок"] = record.get("Заголовок")

    return metadata

loader = JSONLoader(
    file_path=JSON_SAVE_PATH,
    jq_schema='.[]', # схема
    text_content=False, 
    metadata_func=metadata_func, 
    content_key="text" # главный столбец для анализа
)

documents = loader.load()

CPU times: user 48.6 ms, sys: 10.4 ms, total: 59 ms
Wall time: 745 ms


In [15]:
documents[1]

Document(page_content='Российские юристы обсудили спорные моменты в законодательстве - Российская газетаНовостиНовости компанийНаши проектыСвежий номерРГ-НеделяРодинаТематические приложенияСоюзСвежий номерПодписка на изданиеПодписаться на новостиВойтиНовостиВластьЭкономикаВ регионахВ миреПроисшествияОбществоСпортКультураДокументыСпецпроектыЕщёЛента новостейДокументыСпецпроектыВласть13.10.2014 23:44ПоделитьсяРоссийские юристы обсудили спорные моменты в законодательствеИрина КолодинаВ этом году в России произошло несколько важных событий, имеющих далеко идущие юридические последствия. В марте в состав Российской Федерации вошли два новых субъекта - Республика Крым и город Севастополь. Сейчас в этих субъектах - переходный период, в ходе которого формируются суды, система адвокатуры и нотариата, решаются вопросы кадастра и регистрации прав на недвижимость. 6 августа начал работу обновленный Верховный суд РФ, который теперь занимается и рассмотрением экономических споров. И наконец, летом и

In [17]:
#  разбиваем элемент из documents на более мелкие части

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)
texts[0:5]

[Document(page_content='С 1 июня в силу вступят новые поправки в Гражданский кодекс РФ - Российская газетаНовостиНовости компанийНаши проектыСвежий номерРГ-НеделяРодинаТематические приложенияСоюзСвежий номерПодписка на изданиеПодписаться на новостиВойтиНовостиВластьЭкономикаВ регионахВ миреПроисшествияОбществоСпортКультураДокументыСпецпроектыЕщёЛента новостейДокументыСпецпроектыВласть26.05.2015 18:05ПоделитьсяС 1 июня в силу вступят новые поправки в Гражданский кодекс РФИрина ГоловаНа 2015 год пришелся большой объем', metadata={'source': 'https://rg.ru/2015/05/27/popravki.html', 'seq_num': 1}),
 Document(page_content='РФИрина ГоловаНа 2015 год пришелся большой объем изменений в Гражданский кодекс РФ в рамках реформы гражданского законодательства. Причем поправки вступили в силу не только в январе, но и в марте, и в апреле. Большой блок подготовлен к введению с 1 июня 2015 года.Одно из значимых изменений - с 1 января 2015 года исчезла необходимость описывать все имущество, находящееся в

In [23]:
embeddings = GigaChatEmbeddings(
    credentials=CREDS, scope='GIGACHAT_API_CORP', verify_ssl_certs=False
)

time: 435 µs (started: 2024-04-24 20:15:18 +03:00)


In [27]:
db_faiss = FAISS.from_documents(texts, embeddings)

time: 4.38 s (started: 2024-04-24 20:32:40 +03:00)


# Как работает

In [30]:
query = 'Какие поправки вступили в Гражданский кодекс'
db_faiss.similarity_search_with_score(query, k=5)

[(Document(page_content='С 1 июня в силу вступят новые поправки в Гражданский кодекс РФ - Российская газетаНовостиНовости компанийНаши проектыСвежий номерРГ-НеделяРодинаТематические приложенияСоюзСвежий номерПодписка на изданиеПодписаться на новостиВойтиНовостиВластьЭкономикаВ регионахВ миреПроисшествияОбществоСпортКультураДокументыСпецпроектыЕщёЛента новостейДокументыСпецпроектыВласть26.05.2015 18:05ПоделитьсяС 1 июня в силу вступят новые поправки в Гражданский кодекс РФИрина ГоловаНа 2015 год пришелся большой объем', metadata={'source': 'https://rg.ru/2015/05/27/popravki.html', 'seq_num': 1}),
  233.34628),
 (Document(page_content='Президент РФ Владимир Путин подписал\xa0очередной блок поправок\xa0в Гражданский кодекс РФ, касающийся\xa0залога и уступки прав\xa0требования. Сегодня этот 83-страничный документ размещен на портале официального опубликования правовых актов. Федеральным законом в новой редакции излагается\xa0параграф 3 "Залог" главы 23\xa0части первой ГК РФ.\xa0Уточняются 

time: 583 ms (started: 2024-04-24 20:34:41 +03:00)


In [31]:
query = 'штрафы за отказ от исполнения обязательств'
db_faiss.similarity_search_with_score(query, k=5)

[(Document(page_content='следующие последствия: при недействительности основного обязательства недействительным становится и залоговое обязательство, уступка прав по основному обязательству влечёт за собой уступки прав по залоговому и\xa0т.\xa0д. Залог недвижимого имущества носит название ипотека.', metadata={'source': 'https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%BB%D0%BE%D0%B3_(%D0%B3%D1%80%D0%B0%D0%B6%D0%B4%D0%B0%D0%BD%D1%81%D0%BA%D0%BE%D0%B5_%D0%BF%D1%80%D0%B0%D0%B2%D0%BE)', 'seq_num': 8}),
  241.97705),
 (Document(page_content='на залог имущества определенных рода или вида. Например, договоре о залоге может содержать такую фразу: "Залогом является все имущество, принадлежащее залогодателю на праве собственности на момент заключения настоящего договора". Очевидно, что данная конструкция снизит вероятность оспаривания залога со стороны недобросовестных должников. Между тем, у этой новеллы есть и недостатки. "При всей моей любви к кредиторам и нелюбви к должникам, которые не исполня

time: 1.66 s (started: 2024-04-24 20:42:20 +03:00)


Тут только 5-е в тему.

Следующие шаги:
- применить разные способы деления на чанки
- проверить эмбеддинги мультилингв модели
- можно попробовать другую векторную базу (chroma)
- улучшить сам датасет, например, добавить столбец "Заголовок", почистить склеенные слова или лишние куски.